In [6]:
from Bio import SeqIO, Entrez
import os
from urllib.error import HTTPError
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import permutations, product
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import tqdm
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_val_score
import inspect
from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
from sklearn.metrics import accuracy_score, auc, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc, roc_curve, roc_auc_score

from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier

from sklearn.model_selection import GridSearchCV

if (os.path.abspath('').split('/')[-1] == 'project'):
    %cd utils
elif (os.path.abspath('').split('/')[-1] == 'validate'):
    %cd ../utils

import query_utils
import model_utils
import validation_utils
import data_utils


print(os.path.abspath(''))

if (os.path.abspath('').split('/')[-1] == 'utils'):
    %cd ..

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

Entrez.tool = "Zoonosis predictor"

Entrez.email = input("Enter an email address to use NCBI e-utils: ")

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [17]:
def getSequences(mergedDf):
    accession_list = [] # maintain order
    isZoonotic_list = [] # maintain order
    accession_set = set()
    isZoonotic_set = set()


    for row in tqdm.tqdm(mergedDf.itertuples()):
        # row[13] = accession, row[15] = infects humans
        for single_acc in row[14].split("; "):
            # print(single_acc)
            if single_acc not in accession_set:
                accession_list.append(single_acc)
                isZoonotic_list.append(0 if not row[16] else 1)
                # accession_set.add(single_acc)
                # isZoonotic_set.add(row[15])
                # print(0 if not row[16] else 1)

    print("passed local retrieval")
    # TODO: RUN MULTIPLE THREADS TO SPEED UP
    threads = []
    vals = []

    
    for index, ID in enumerate(tqdm.tqdm(accession_list[:1000])): #only read the first 100 lol
        # multithread for speed up
        query_utils.queryKmer(ID, isZoonotic_list, index, vals)
        # x = threading.Thread(target=queryKmer, args=(ID, isZoonotic_list, index, vals))
        # threads.append(x)
        # x.start()

    # for index, thread in enumerate(tqdm.tqdm(threads)):
    #     thread.join()
    df = pd.DataFrame(vals)
    df.to_csv("data/nardus_sequences.csv")

    return df
    


In [7]:
def getSingleSequence(accessionID):
    try:
        QueryHandle = Entrez.efetch(db="nucleotide", id=accessionID, 
                                    rettype="gb", retmode="text")
    except HTTPError as Error:
        if Error.code == 400:  # Bad request
            raise ValueError(f"Accession number not found: {accessionID}")
        else:
            raise

    SeqRec = SeqIO.read(QueryHandle, "genbank")
    print(SeqRec.seq)
    X_info = SeqRec.seq.lower()
    kmer = 4
    s = product('acgt',repeat = kmer)
    permset = set(["".join(x) for x in list(s)])


    oDict = data_utils.assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

    kmer_df = pd.DataFrame()

    for i in oDict:
        kmer_df.at[0, i]=oDict[i]
    # print(best_gradBoost.predict_proba(kmer_df))
    kmer_df = kmer_df.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)

    model = pickle.load(open('models/curr_models/final_merged_stackingcv.pkl', 'rb'))
    
    # re-align columns
    print(inspect.getmembers(model))
    kmer_df = kmer_df[model.train_meta_features]
    print(model.predict_proba(kmer_df)[:,1])


getSingleSequence("NC_002549.1")
# west nile is ALREADY validated - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5658487/
getSingleSequence("MF797870")
# dengue need to remove
getSingleSequence("NC_001474")
# zika - DOES NOT have?? - check other strains though
getSingleSequence("MG645981")
# # salmonella - too large
# getSingleSequence("CP026379")

# nonzoonotic cattle thing - dataset does not include this?
getSingleSequence("NC_028246.1")

CGGACACACAAAAAGAAAGAAGAATTTTTAGGATCTTTTGTGTGCGAATAACTATGAGGAAGATTAATAATTTTCCTCTCATTGAAATTTATATCGGAATTTAAATTGAAATTGTTACTGTAATCACACCTGGTTTGTTTCAGAGCCACATCACAAAGATAGAGAACAACCTAGGTCTCCGAAGGGAGCAAGGGCATCAGTGTGCTCAGTTGAAAATCCCTTGTCAACACCTAGGTCTTATCACATCACAAGTTCCACCTCAGACTCTGCAGGGTGATCCAACAACCTTAATAGAAACATTATTGTTAAAGGACAGCATTAGTTCACAGTCAAACAAGCAAGATTGAGAATTAACCTTGGTTTTGAACTTGAACACTTAGGGGATTGAAGATTCAACAACCCTAAAGCTTGGGGTAAAACATTGGAAATAGTTAAAAGACAAATTGCTCGGAATCACAAAATTCCGAGTATGGATTCTCGTCCTCAGAAAATCTGGATGGCGCCGAGTCTCACTGAATCTGACATGGATTACCACAAGATCTTGACAGCAGGTCTGTCCGTTCAACAGGGGATTGTTCGGCAAAGAGTCATCCCAGTGTATCAAGTAAACAATCTTGAAGAAATTTGCCAACTTATCATACAGGCCTTTGAAGCAGGTGTTGATTTTCAAGAGAGTGCGGACAGTTTCCTTCTCATGCTTTGTCTTCATCATGCGTACCAGGGAGATTACAAACTTTTCTTGGAAAGTGGCGCAGTCAAGTATTTGGAAGGGCACGGGTTCCGTTTTGAAGTCAAGAAGCGTGATGGAGTGAAGCGCCTTGAGGAATTGCTGCCAGCAGTATCTAGTGGAAAAAACATTAAGAGAACACTTGCTGCCATGCCGGAAGAGGAGACAACTGAAGCTAATGCCGGTCAGTTTCTCTCCTTTGCAAGTCTATTCCTTCCGAAATTGGTAGTAGGAGAAAAGGCTTGCCTTGAGAAGGTTCAAAGGCAAATTCAA

AttributeError: 'StackingCVClassifier' object has no attribute 'train_meta_features'

In [11]:

    # print(kmer_df)
    # best_gradBoost = pickle.load(open('models/nardus_testing/gradBoost.pkl', 'rb'))
    # cols_when_model_builds = best_gradBoost.feature_names_in_
    # kmer_df=kmer_df[cols_when_model_builds]
    
    # print(best_gradBoost.predict_proba(kmer_df)[:,1])
    # other = pickle.load(open('models/curr_models/gradBoost.pkl', 'rb'))
    
    # cols_when_model_builds = other.feature_names_in_
    # kmer_df=kmer_df[cols_when_model_builds]
    
    # print(other.predict_proba(kmer_df)[:,1])

    # asdaf = pickle.load(open('models/curr_models/knn.pkl', 'rb'))
    
    # print(asdaf.predict_proba(kmer_df)[:,1])

    # asdaf = pickle.load(open('models/curr_models/randforest.pkl', 'rb'))
    
    # print(asdaf.predict_proba(kmer_df)[:,1])